In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os
import shutil

if os.path.exists('./file/5.vehicle'):
    shutil.rmtree('./file/5.vehicle')

os.makedirs('./file/5.vehicle')

# = = = = = = = = = = = = = = =

import requests
import json

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

data = {'getAPIKeyForUser': {'catalog': 'tecdocsw',
                             'username': '186299u15',
                             'password': 'APMTeDoW14'}}

b = 0
while True:
    b += 1

    try:
        resp = requests.post('https://webservice.tecalliance.services/webcat30/v1/services/WebCat30WS.jsonEndpoint',
                             data=json.dumps(data),
                             headers=get_header(),
                             #proxies=get_proxy(),
                             timeout=(10, 10))

        if resp.status_code == 200:
            json_ = resp.json()

            if json_['status'] == 200:
                key = json_['apiKey']
                break
    except KeyboardInterrupt:
        break
    except:
        continue

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/4.link'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

for file in list_file:
    
    # = = = = = = = = = = = = = = =

    output_correct, output_error = pd.DataFrame(), pd.DataFrame()

    # = = = = = = = = = = = = = = =

    input_ = pd.read_excel(f'./file/4.link/{file}',
                           header=0,
                           dtype=str).fillna('')
    
    input_['No'] = input_['No'].astype(int)
    input_['Part Code'] = input_['Part Code'].astype(int)
    input_['Make No'] = input_['Make No'].astype(int)
    input_['Make Code'] = input_['Make Code'].astype(int)
    input_['Link No'] = input_['Link No'].astype(int)

    print(f'总数量：{len(input_)}')
    print()

    # = = = = = = = = = = = = = = =


    work = Queue()
    for a in range(len(input_)):
        work.put_nowait(a)

    # = = = = = = = = = = = = = = =

    def crawler():
        global output_correct, output_error, key

        while not work.empty():
            a = work.get_nowait()

            # = = = = = = = = = = = = = = =

            crawler_status = 'error'

            # = = = = = = = = = = = = = = =

            try:
                data = {'getArticleLinkedAllLinkingTargetsByIds3': {'articleCountry': 'DE',
                                                                    'articleId': int(input_.loc[a, 'Part Code']),
                                                                    'immediateAttributs': True,
                                                                    'lang': 'en',
                                                                    'linkedArticlePairs': json.loads(input_.loc[a, 'Json_Link']),
                                                                    'linkingTargetType': 'VOLB',
                                                                    'provider': 40_000}}
                
                b = 0
                while True:
                    b += 1

                    try:
                        resp = requests.post('https://webservice.tecalliance.services/pegasus-3-0/services/TecdocToCatDLB.jsonEndpoint',
                                             data=json.dumps(data),
                                             headers=get_header(**{'x-api-key': key}),
                                             #proxies=get_proxy(),
                                             timeout=(10, 10))
                            
                        if resp.status_code == 200:
                            json_ = resp.json()
                
                            if json_['status'] == 401 and json_['statusText'].strip() == 'Access not allowed':
                                while True:
                                    try:
                                        data = {'getAPIKeyForUser': {'catalog': 'tecdocsw',
                                                                     'username': '186299u15',
                                                                     'password': 'APMTeDoW14'}}
                                        
                                        resp = requests.post('https://webservice.tecalliance.services/webcat30/v1/services/WebCat30WS.jsonEndpoint',
                                                             data=json.dumps(data),
                                                             headers=get_header(),
                                                             #proxies=get_proxy(),
                                                             timeout=(10, 10))
                                
                                        if resp.status_code == 200:
                                            json_ = resp.json()
                                
                                            if json_['status'] == 200:
                                                key = json_['apiKey']
                                                break
                                    except KeyboardInterrupt:
                                        break
                                    except:
                                        continue
                            elif json_['status'] == 200:
                                break
                    except KeyboardInterrupt:
                        break
                    except:
                        continue

                # = = = = = = = = = = = = = = =

                list_vehicle = json_['data']['array']

                # = = = = = = = = = = = = = = =

                if not list_vehicle:
                    raise

                # = = = = = = = = = = = = = = =

                df_temp = pd.DataFrame()
                for i, vehicle in zip(range(len(list_vehicle)), list_vehicle):
                    dict_ = {'Vehicle No': i+1}
                    list_dict_attr = [] if vehicle['linkedArticleImmediateAttributs'] == '' else vehicle['linkedArticleImmediateAttributs']['array']
                    for dict_attr in list_dict_attr:
                        if 'attrValue' in dict_attr:
                            if dict_attr['attrIsConditional']:
                                print(f'''fuck attrIsConditional {input_.loc[a, 'Part Number']}''')
                            elif 'attrShortName' in dict_attr and dict_attr['attrShortName'] != dict_attr['attrName']:
                                print(f'''fuck attrShortName {input_.loc[a, 'Part Number']}''')
                            else:
                                dict_[dict_attr['attrName'].strip()] = dict_attr['attrValue'].strip()
                
                    if len(vehicle['linkedVehicles']['array']) != 1:
                        print(f'''fuck linkedVehicles {input_.loc[a, 'Part Number']}''')
                    df_temp = pd.concat([df_temp, pd.DataFrame([dict(input_.iloc[a]) | dict_ | vehicle['linkedVehicles']['array'][0]])], ignore_index=True).fillna('')
                
                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =

                crawler_status = 'ok'
            except:
                output_error = pd.concat([output_error, pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =

            if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and work.qsize() == 0 or work.qsize() % 1_000 == 0 and work.qsize() != 0:
                clear_output()
                pass

            print(f'''[状态：{crawler_status}，尝试次数：{b}] - {(list_file.index(file) + 1) / len(list_file) * 100:.2f}% - {file.removesuffix('.xlsx')} > {input_.loc[a, 'No']}.{input_.loc[a, 'Part Number']} > {input_.loc[a, 'Make No']}.{input_.loc[a, 'Make']} > Link {input_.loc[a, 'Link No']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

    # = = = = = = = = = = = = = = =

    list_task = []
    for _ in range(50):
        task = gevent.spawn(crawler)
        list_task.append(task)
    gevent.joinall(list_task)

    print('输出ing...')
    print()
    if not output_correct.empty:
        output_correct = output_correct.sort_values(by=['No', 'Make No', 'Link No', 'Vehicle No'],
                                                    ascending=[True, True, True, True],
                                                    ignore_index=True)
        output_correct.to_excel(f'''./file/5.vehicle/{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
    if not output_error.empty:
        output_error = output_error.sort_values(by=['No', 'Make No', 'Link No'],
                                                ascending=[True, True, True],
                                                ignore_index=True)
        output_error.to_excel(f'''./file/{file.removesuffix('.xlsx')}-vehicle_error.xlsx''', index=False)
        print('爬虫存在error')
        print()

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/5.vehicle'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

from tqdm import tqdm
import re

for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'./file/5.vehicle/{file}',
              f'''./file/5.vehicle/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)}''')

print('Done ~')

总数量：6974

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 4.01107 > 2.VW > Link 2
[剩余数量：6924] - [当前时间：16:09:32]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 6.01267 > 1.MAN > Link 3
[剩余数量：6923] - [当前时间：16:09:32]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 6.01267 > 1.MAN > Link 14
[剩余数量：6922] - [当前时间：16:09:32]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 6.01267 > 1.MAN > Link 19
[剩余数量：6921] - [当前时间：16:09:32]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 4.01107 > 2.VW > Link 5
[剩余数量：6920] - [当前时间：16:09:32]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 6.01267 > 1.MAN > Link 16
[剩余数量：6919] - [当前时间：16:09:32]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 6.01267 > 1.MAN > Link 22
[剩余数量：6918] - [当前时间：16:09:32]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 4.01107 > 2.VW > Link 4
[剩余数量：6917] - [当前时间：16:09:32]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 5.01109 > 2.VW > Link 2
[剩余数量：6916] - [当前时间：16:09:32]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 6.01267 > 1.MAN > Link 13
[剩余数量：6915] - [当前时间：16:09:32]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 4.01107 > 1

Progress: 100%|██████████████████████████████| 2/2 [00:00<00:00, 3682.44it/s]

Done ~
